# Visualize test performance with Confusion Matrix

In [1]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import argparse
from easydict import EasyDict as edict

import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import transforms

from datetime import datetime
from sklearn.metrics import confusion_matrix, accuracy_score
from tqdm import tqdm

from dataset import NTUDataset, get_train_val_set
from model import ConvLSTM


def get_train_val_loader(params, val_pct=0.2):
    train_samples, val_samples = get_train_val_set(data_path=params.data_path, val_pct=val_pct, temporal_aug_k=params.temporal_aug_k)
    print(f'Train samples: {len(train_samples)} || Validation samples: {len(val_samples)}')
    
    # Apply transform to normalize the data
    # transform = transforms.Normalize((0.5), (0.5))
    
    # Load train and validation dataset
    train_set = NTUDataset(sample_set=train_samples, params=params, transform=None)
    val_set = NTUDataset(sample_set=val_samples, params=params, transform=None)

    train_loader = DataLoader(train_set, batch_size=params.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=params.BATCH_SIZE, shuffle=True)
    
    return train_loader, val_loader


def save_model(model):
    current_time = datetime.now()
    current_time = current_time.strftime("%m_%d_%Y_%H_%M")
    torch.save(model.state_dict(), f'../saved_models/ntu_lstm_{current_time}.pth')
    
    
def build_test_stats(preds, actual, acc, params):
    print(f'Model accuracy: {acc}')
    
    # For confusion matrix
    preds = [int(k) for k in preds]
    actual = [int(k) for k in actual]

    cf = confusion_matrix(actual, preds, labels=list(range(params.num_classes)))
    return cf


def train(model, train_loader, loss_function, optimizer, params):
    print('Training...')
    for epoch in range(params.n_epochs):
        for batch in tqdm(train_loader):
            inputs, labels = batch[0].to(device).float(), batch[1].to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()

        print(f'Epoch: {epoch} | Loss: {loss}')

    return model

def test(model, test_loader):
    print('Testing...')
    correct = 0
    total = 0

    preds = []
    actual = []

    with torch.no_grad():
        for batch in tqdm(test_loader):
            inputs, labels = batch
            inputs, labels = inputs.to(device).float(), labels.to(device)
            class_outputs = model(inputs)
            _, class_prediction = torch.max(class_outputs.data, 1)
            total += labels.size(0)
            correct += (class_prediction == labels).sum().item()
            preds.extend(list(class_prediction.to(dtype=torch.int64)))
            actual.extend(list(labels.to(dtype=torch.int64)))

    acc = 100*correct/total
    return preds, actual, acc


def main(params):
    # Initialize some variables to track progress
    accs = []
    
    # Initialize the model
    model = ConvLSTM(params=params).to(device)
    
    # Use parallel computing if available
    if device.type == 'cuda' and n_gpus > 1:
        model = nn.DataParallel(model, list(range(n_gpus)))
        
    # Loss Function and Optimizer (can use weight=class_weights if it is a disbalanced dataset)
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Get train and validation loaders
    train_loader, val_loader = get_train_val_loader(params, val_pct=0.2)
    
    # Train the model
    model = train(model, train_loader, loss_function, optimizer, params)
    save_model(model)

    # Get training accuracy
    preds, actual, acc = test(model, train_loader)
    build_test_stats(preds, actual, acc, params)
    
    # Validate the model
    preds, actual, acc = test(model, val_loader)
    build_test_stats(preds, actual, acc, params)
    

## Optional code to load and test a model
def load_test_model(params, model_path):
    model = ConvLSTM(params=params).to(device)
    # Use this to fix keyError in the model when using DataParallel while training
    if device.type == 'cuda' and n_gpus > 1:
        model = nn.DataParallel(model, list(range(n_gpus)))
    train_loader, val_loader = get_train_val_loader(params, val_pct=0.2)
    model.load_state_dict(torch.load(model_path))
    model.eval() # To set dropout and batchnormalization OFF
    preds, actual, acc = test(model, val_loader)
    return build_test_stats(preds, actual, acc, params)

In [6]:
params = {'mode': 'inference', 'model_path': '/home/youngjoon/DEV/ntu-skeleton/code/saved_models/ntu_lstm_10_30_2022_14_34.pth', 'kp_shape': [18, 3], 'seg_size': 50, 
          'data_path': '/home/youngjoon/DEV/NTU_VIBE_NPY(Raw)/', 
          'BATCH_SIZE': 8, 'temporal_aug_k': 3, 'k_fold': 1, 'n_epochs': 20, 'num_classes': 13, 'bcc': 32, 'num_channels': 1,
          'num_joints': 18, 'num_coord': 3}
params = edict(params)
# Check for GPUs
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
n_gpus = torch.cuda.device_count()
print(f'Number of GPUs available: {n_gpus}')

Number of GPUs available: 1


In [7]:
cf = load_test_model(params, model_path=params.model_path)
df_cm = pd.DataFrame(cf, index=[str(i) for i in range(1, 14)], columns=[str(i) for i in range(1, 14)])
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

Total Video Samples: 1191 || Total Participants: 104 || Train Participants: 84 || Validation Participants: 20
Train samples: 2952 || Validation samples: 621
Evaluating mean and std for the training set...


ValueError: cannot reshape array of size 3750 into shape (50,1,18,3)